### Prelims

#### Imports

In [ ]:
# Standard libraries
import os
from typing import Any, Sequence
import numpy as np

%load_ext tensorboard

# JAX/Equinox
import jax
import jax.numpy as jnp
from jax import random
import equinox as eqx

# PyTorch for data loading
import torch
import torch.utils.data as data

## Imports for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## equinox helper funcitons
from eqx_helpers import EqxTrainerModule, create_data_loaders

sns.reset_orig()
sns.set_theme()

DATASET_PATH = os.path.join(os.getcwd(), "data")
CHECKPOINT_PATH = os.path.join(os.getcwd(), "saved_models\\cifar_mess\\")

#### Load Data

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

# Transformations applied on each image => bring them into a numpy array
DATA_MEANS = np.array([0.49139968, 0.48215841, 0.44653091])
DATA_STD = np.array([0.24703223, 0.24348513, 0.26158784])


def image_to_numpy(img):
    img = np.array(img, dtype=np.float32)
    img = (img / 255.0 - DATA_MEANS) / DATA_STD
    return img.transpose(2,0,1)


def img_restore(img):
    img = img.transpose(1,2,0)
    img = ((img * DATA_STD) + DATA_MEANS) * 255.0
    img = np.array(img, dtype=np.int32)
    return img

# augmentation step
test_transform = transforms.Compose(
    [   image_to_numpy,
    ]
)
train_transform = transforms.Compose(
    [   transforms.TrivialAugmentWide(),
        image_to_numpy,
    ]
)

# we don't want to augment the validation data, so we split this up
train_dataset = CIFAR10(
    root=DATASET_PATH, train=True, transform=train_transform, download=True
)
val_dataset = CIFAR10(
    root=DATASET_PATH, train=True, transform=test_transform, download=True
) # note that we use `test_transform here`


# we split with the same seed, but using the transformed and untransformed data separately
train_set, _ = data.random_split(
    train_dataset, [45000, 5000], generator=torch.Generator().manual_seed(42)
)
_, val_set = data.random_split(
    val_dataset, [45000, 5000], generator=torch.Generator().manual_seed(42)
)

# Loading the test set
test_set = CIFAR10(
    root=DATASET_PATH, train=False, transform=test_transform, download=True
)

#reducing the size of the data by 96% for testing:
_, train_set = data.random_split(
    train_set, [43200, 1800], generator=torch.Generator().manual_seed(42)
)

_, val_set = data.random_split(
    val_set, [4800, 200], generator=torch.Generator().manual_seed(42)
)
_, test_set = data.random_split(
    test_set, [9600, 400], generator=torch.Generator().manual_seed(42)
)



train_loader, val_loader, test_loader = create_data_loaders(
    train_set, val_set, test_set, train=[True, False, False], batch_size= 200 #256
)

### CNN model

#### define model, blocks

In [ ]:
class NormedConv_Block(eqx.Module):
    conv: eqx.nn.LayerNorm
    pool: eqx.nn.MaxPool2d
    drop: eqx.nn.Dropout
    batch_norm: eqx.nn.LayerNorm

    def __init__(
        self,
        input_shape: Sequence[int] = (3,32,32),
        out_channels: int = 16,
        conv_window: int = 3,
        pooling_kernel: int = 2,
        pooling_stride: int = 1,
        drop_rate: int = 0.0,
        key: jax.Array = random.key(42),
    ):
        """Block with the following layers:
        2D convolution -> 2D pooling -> RELU activation -> batch normalization

        Args:
            input_shape (Sequence[int], optional): Shape of (unbatched) input. Order is Channel, Height, Width. Defaults to (3,32,32).
            out_channels (int, optional): Number of output channels. Defaults to 16.
            conv_window (int, optional): Size of 2d convolutional window. Defaults to 3.
            pooling_kernel (int, optional): Kernel for 2d max pooling. Defaults to 2.
            pooling_stride (int, optional): Stride length for pooling. Defaults to 1.
            drop_rate (int, optional): Dropout rate. Defaults to 0.0.
            key (jax.Array, optional): PRNGKey. Defaults to random.key(42).
        """            
        self.conv = eqx.nn.Conv2d(
            input_shape[0], out_channels, kernel_size=conv_window, key=key
        )
        if pooling_kernel == pooling_stride == 1:
            self.pool = eqx.nn.Identity()
        else:
            self.pool = eqx.nn.MaxPool2d(pooling_kernel, stride=pooling_stride)
        self.drop = eqx.nn.Dropout(drop_rate)

        new_H = (
            int(((input_shape[1] - conv_window) + 1 - pooling_kernel) / pooling_stride)
            + 1
        )
        input_shape = (out_channels, new_H, new_H)
        self.batch_norm = eqx.nn.BatchNorm(out_channels, "batch", momentum=0.5)
        # self.batch_norm = eqx.nn.LayerNorm(input_shape[1:])

    def __call__(self, x, key, state, train=True, return_activations=False):
        activations= []
        x = self.conv(x)
        activations.append(x)
        x = self.pool(x)
        x = jax.nn.gelu(x)
        x = self.drop(x, key=key, inference=not train)
        # x, state = self.batch_norm(x, state, inference=not train)
        # x = jax.vmap(self.batch_norm)(x)
        return (x, state, activations) if return_activations else (x, state)


class NormedDense_Block(eqx.Module):
    dense: eqx.nn.Linear
    drop: eqx.nn.Dropout
    batch_norm: eqx.nn.BatchNorm

    def __init__(self, 
                 in_dim: int = 64, 
                 out_dim: int = 10, 
                 drop_rate: float = 0.0,
                 key: jax.Array = random.key(42)):
        """Block with the following layers:
        Linear NN -> RELU activation -> layer normalization

        Args:
            in_dim (int, optional): (Unbatched) input dimension. Defaults to 64.
            out_dim (int, optional): (Unbatched) output dimension. Defaults to 10.
            drop_rate (float, optional): Dropout rate. Defaults to 0.0.
            key (jax.Array, optional): PRNGKey. Defaults to random.key(42).
        """        
        self.dense = eqx.nn.Linear(in_dim, out_dim, key=key)
        self.drop = eqx.nn.Dropout(drop_rate)
        # self.batch_norm = eqx.nn.BatchNorm(out_dim, "batch")
        self.batch_norm = eqx.nn.LayerNorm(out_dim)

    def __call__(self, x, key, state, train=True, return_activations=False):
        activations = []
        x = self.dense(x)
        activations.append(x)
        x = jax.nn.gelu(x)
        x = self.drop(x, key=key, inference=not train)
        # x, state = self.batch_norm(x, state, inference=not train)
        # x = self.batch_norm(x)
        return (x, state, activations) if return_activations else (x, state)

In [ ]:
class eqx_CNN_Classifier(eqx.Module):
    conv_layers: Sequence[NormedConv_Block]
    linear_layers: Sequence[NormedDense_Block]

    def __init__(
        self,
        input_shape: Sequence[int] = (3, 32, 32),
        num_classes: int = 10,
        conv_channels: Sequence[int] = [16],
        conv_windows: Sequence[int] = [5],
        pooling_kernel: Sequence[int] = [2],
        pooling_stride: Sequence[int] = [2],
        conv_drop: float = 0.0,
        hidden_dims: Sequence[int] = [64],
        dense_drop: float = 0.0,
        key: jax.Array = random.key(42),
    ):
        """Classifier formed by stacking convolutional and dense blocks.

        Args:
            input_shape (Sequence[int], optional): (Unbatched) shape of input. Order is Channel, Height, Width. Defaults to (3, 32, 32).
            num_classes (int, optional): Number of classes to predict. Shape of final dense output. Defaults to 10.
            conv_channels (Sequence[int], optional): Number of channels for each convolutional layer. Defaults to [16].
            conv_windows (Sequence[int], optional): Window size for each convolutional layer. Defaults to [5].
            pooling_kernel (Sequence[int], optional): Kernel size for each convolutional layer. Defaults to [2].
            pooling_stride (Sequence[int], optional): Stride length for each convolutional layer. Defaults to [2].
            conv_drop (float, optional): Dropout rate for convolutional layers. Defaults to 0.0.
            hidden_dims (Sequence[int], optional): Hidden dims for intermediate dense layers. Defaults to [64].
            dense_drop (float, optional): Dropout for dense layers. Defaults to 0.0.
            key (jax.Array, optional): PRNGKey. Defaults to random.key(42).
        """        
        def assert_equal_length(*sequences):
            # Calculate the lengths of all sequences
            lengths = [len(seq) for seq in sequences]

            # Check if all lengths are equal to the first length in the list
            assert all(
                length == lengths[0] for length in lengths
            ), f"Sequences defining ConvSA block are of different lengths: {lengths}"

        assert_equal_length(conv_channels, conv_windows, pooling_kernel, pooling_stride)
        
        # establish convulutional layers
        self.conv_layers = []
        key, *conv_keys = random.split(key, num=len(conv_channels) + 1)
        conv_iterator = enumerate(
            zip(conv_windows, conv_channels, pooling_kernel, pooling_stride, conv_keys)
        )
        for i, (cw, cf, pk, ps, k) in conv_iterator:
            self.conv_layers.append(
                NormedConv_Block(
                    input_shape=input_shape,
                    out_channels=cf,
                    conv_window=cw,
                    pooling_kernel=pk,
                    pooling_stride=ps,
                    key=k,
                    drop_rate=conv_drop,
                )
            )

            new_H = (
                int(
                    ((input_shape[1] - conv_windows[i]) + 1 - pooling_kernel[i])
                    / pooling_stride[i]
                )
                + 1
            )
            input_shape = (cf, new_H, new_H)

        # establish linear layers
        self.linear_layers = []
        key, *lin_keys = random.split(key, num=len(hidden_dims) + 1)
        input_shape = np.prod(input_shape)
        for i, (hidden_dim, k) in enumerate(zip(hidden_dims, lin_keys)):
            self.linear_layers.append(
                NormedDense_Block(
                    in_dim=input_shape, out_dim=hidden_dim, key=k, drop_rate=dense_drop
                )
            )
            input_shape = hidden_dim
        self.linear_layers.append(
            NormedDense_Block(
                in_dim=input_shape, out_dim=num_classes, key=key, drop_rate=0
            )
        )

    def __call__(self, x, key, state, train=True, return_activations = False):
        subkey, *conv_keys = random.split(key, num=len(self.conv_layers) + 1)
        activations = []
        for layer, k in zip(self.conv_layers, conv_keys):
            outputs = layer(x, k, state=state, train=train, return_activations=return_activations)
            x, state = outputs[0], outputs[1]
            if return_activations:
                activations += outputs[2]
        x = jnp.ravel(x)
        subkey, *lin_keys = random.split(subkey, num=len(self.linear_layers) + 1)
        for layer, k in zip(self.linear_layers, lin_keys):
            outputs = layer(x, k, state=state, train=train, return_activations=return_activations)
            x, state = outputs[0], outputs[1]
            if return_activations:
                activations += outputs[2]
        x = jax.nn.log_softmax(x)
        return (x, state, activations) if return_activations else (x, state)

#### Set up trainer

In [ ]:
import optuna


class CNNClassTrainer(EqxTrainerModule):
    def __init__(
        self,
        conv_windows: Sequence[int],
        conv_channels: Sequence[int],
        pooling_kernel: Sequence[bool],
        pooling_stride: Sequence[bool],
        hidden_dims: Sequence[int],
        input_shape: int,
        num_classes: int,
        conv_drop: float = 0.0,
        dense_drop: float = 0.0,
        trial: Any = None,
        **kwargs,
    ):
        self.trial = trial
        super().__init__(
            model_class=eqx_CNN_Classifier,
            model_hparams={
                "conv_windows": conv_windows,
                "conv_channels": conv_channels,
                "pooling_kernel": pooling_kernel,
                "pooling_stride": pooling_stride,
                "input_shape": input_shape,
                "hidden_dims": hidden_dims,
                "num_classes": num_classes,
                "conv_drop": conv_drop,
                "dense_drop": dense_drop,
            },
            **kwargs,
        )
    def model_init(self, key):
        def init_linear_weight(model, key):
            # initializer = jax.nn.initializers.variance_scaling(2., "fan_", 'truncated_normal', -1, -2)
            initializer = jax.nn.initializers.xavier_normal(-1, -2)
            def replace(x):
                return isinstance(x, eqx.nn.Linear)
            def get_weights(m):
                return [x.weight for x in jax.tree_util.tree_leaves(m, is_leaf=replace) if replace(x)]
            weights = get_weights(model)
            new_weights = [initializer(subkey, weight.shape)
                            for weight, subkey in zip(weights, jax.random.split(key, len(weights)))]
            new_model = eqx.tree_at(get_weights, model, new_weights)
            return new_model
        def init_conv_weight(model, key):
            # initializer = jax.nn.initializers.variance_scaling(2., "fan_in", 'truncated_normal', -3, -4)
            initializer = jax.nn.initializers.xavier_normal(-3, -4)
            def replace(x):
                return isinstance(x, eqx.nn.Conv2d)
            def get_weights(m):
                return [x.weight for x in jax.tree_util.tree_leaves(m, is_leaf=replace) if replace(x)]
            weights = get_weights(model)
            new_weights = [initializer(subkey, weight.shape)
                            for weight, subkey in zip(weights, jax.random.split(key, len(weights)))]
            new_model = eqx.tree_at(get_weights, model, new_weights)
            return new_model
        key1, key2 = random.split(key)
        self.model = init_linear_weight(self.model, key1)
        self.model = init_conv_weight(self.model, key2)

    def create_functions(self):
        def loss_function(model, x, y, key, state, train=True):
            model_out, state = jax.vmap(
                model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, key, state, train)
            # since we use log_softmax, cross entropy is just the value of our model output
            # indexed by the true value of y.
            loss = jnp.take_along_axis(model_out, jnp.expand_dims(y, 1), axis=1)
            loss = -jnp.mean(loss)

            pred_y = jnp.argmax(model_out, axis=1)
            acc = jnp.mean(y == pred_y)
            return loss, (acc, state)

        def train_step(model, opt_state, batch, key, state):
            x, y = batch
            (loss, (acc, state)), grads = eqx.filter_value_and_grad(
                loss_function, has_aux=True
            )(model, x, y, key, state)
            updates, opt_state = self.opt.update(grads, opt_state, model)
            model = eqx.apply_updates(model, updates)
            metrics = {"loss": loss, "acc": acc}
            return model, opt_state, metrics, state

        def eval_step(model, batch, key, state):
            x, y = batch
            _, (acc, _) = loss_function(model, x, y, key, state, train=False)
            return {"acc": acc}

        return train_step, eval_step

    def on_validation_epoch_end(self, epoch_idx, eval_metrics, val_loader):
        if self.trial:
            self.trial.report(eval_metrics["val/acc"], step=epoch_idx)
            if self.trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    def bind_model(self):
        """
        Returns a batched model with parameters bound to it. Enables an easier inference
        access.

        Returns:
          The model with `key`, `state`, `train`, baked in, such that it only takes x input
        """

        def mod(x):
            model_out, state = jax.vmap(
                self.model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, self.key, self.state, False)
            return model_out

        return mod

    def vizualize_grads(self, batch):
        """Plots histogram of gradients at model leaves. 
            Interior functions are 75% copy-paste from `create_functions`
        """

        def loss_function(model, x, y, key, state, train=True):
            model_out, state = jax.vmap(
                model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, key, state, train)
            # since we use log_softmax, cross entropy is just the value of our model output
            # indexed by the true value of y.
            loss = jnp.take_along_axis(model_out, jnp.expand_dims(y, 1), axis=1)
            loss = -jnp.mean(loss)

            pred_y = jnp.argmax(model_out, axis=1)
            acc = jnp.mean(y == pred_y)
            return loss, (acc, state)

        def get_grad_and_path(model, batch, key, state):
            x, y = batch
            (loss, (acc, state)), grads = eqx.filter_value_and_grad(
                loss_function, has_aux=True
            )(model, x, y, key, state)
            dt = jax.tree_util.tree_leaves_with_path(grads)
            grads, names = [], []
            for (
                path,
                grad,
            ) in (
                dt
            ):  # filter basic info we don't care about (bias terms, normalization)
                if (
                    (len(grad.shape) > 1)
                    and (path[-1].name != "bias")
                    and (not path[-2].name.endswith("norm"))
                ):
                    grads.append(grad.reshape(-1))
                    names.append(path[-2].name + "_" + path[-1].name)
            return grads, names
        
        def viz_grads(grads, names, color="C0"):
            columns = len(grads)
            fig, ax = plt.subplots(1, columns, figsize=(columns * 3.5, 2.5))
            for g_idx, g in enumerate(grads):
                key = f"{names[g_idx]}"
                key_ax = ax[g_idx % columns]
                sns.histplot(data=g, bins=30, ax=key_ax, color=color, kde=False, stat='probability')
                key_ax.set_title(str(key))
                key_ax.set_xlabel("Grad magnitude")
            fig.suptitle("Gradient magnitude distributions", fontsize=14, y=1.05)
            fig.subplots_adjust(wspace=0.45)
            plt.show()
            plt.close()

        grads, names = get_grad_and_path(self.model, batch, self.key, self.state)
        viz_grads(grads, names)
    def visualize_activations(self, batch):
        def get_activations(model, batch, key, state):
            x, y = batch
            preds, state, acts = jax.vmap(
                model,
                axis_name="batch",
                in_axes=(0, None, None, None, None),
                out_axes=(0, None, 0),
            )(x, key, state, False, True)
            acts = [jnp.reshape(a, (-1,)) for a in acts]
            return acts
        def viz_acts(acts, color="C0"):

            nrowsfn = lambda x: x // 4 + 1 if x % 4 != 0 else x // 4  # noqa: E731
            fig, axes = plt.subplots(nrowsfn(len(acts)), 4, sharey=True)
            ax = axes.ravel()
            for i, g in enumerate(acts):
                Q1 = jnp.quantile(g, .25)
                Q3 = jnp.quantile(g, .75)
                IQR = Q3 - Q1
                filtered = g[(g >= (Q1 - 2. * IQR)) & (g <= (Q3 + 2. * IQR))]

                key_ax = ax[i]
                key_ax.set_title(f'var: {jnp.var(g):.2f}')
                sns.histplot(data=filtered, bins=30, ax=key_ax, color=color, kde=False, stat='probability')
            fig.suptitle("Activation magnitude distributions",)
            plt.show()
            plt.close()
        acts = get_activations(self.model, batch, self.key, self.state)
        viz_acts(acts)

In [ ]:
trainer = CNNClassTrainer(
    conv_windows=[3, 5],
    conv_channels=[16, 32],
    pooling_kernel=[2, 2],
    pooling_stride=[2, 2],
    hidden_dims=[64],
    input_shape=(3, 32, 32),
    num_classes=10,
    conv_drop=0.0,
    dense_drop=0.0,
    optimizer_hparams={"optimizer": "nadamw", "lr": 5e-3},
    logger_params={
        "base_log_dir": CHECKPOINT_PATH, # model params, checkpointed models, and TensorBoard events files go here 
    },
    check_val_every_n_epoch=5, # this means the model is checkpointed every fifth epoch
    seed=40,
    debug=False,
)


#### Train Model

In [ ]:
metrics = trainer.train_model(train_loader, val_loader, test_loader, 50)

#### Model Evaluation

In [ ]:
print(f'Validation accuracy: {metrics["val/acc"]:4.2%}')
print(f'Test accuracy: {metrics["test/acc"]:4.2%}')

In [ ]:
batch = next(iter(train_loader))
trainer.vizualize_grads(batch)

In [ ]:
trainer.visualize_activations(batch)

In [ ]:
# confirm loading checkpoints works properly!

imgs, y = batch
mod = trainer.bind_model()

trainer2 = CNNClassTrainer.load_from_checkpoint(os.path.join(CHECKPOINT_PATH, r'eqx_CNN_Classifier\\version_0') )
mod2 = trainer2.bind_model()

pred = mod(imgs).argmax(-1) # our model ends with log-softmax
pred2 = mod2(imgs).argmax(-1)

assert jnp.all(pred2 == pred)

In [ ]:
print((pred != y).mean())
misclass_imgs = imgs[pred != y]
bad_pred = pred[pred != y]
real_answer = y[pred != y]

labels = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [ ]:
i = 2
print(f"predicted: {labels[bad_pred[i]]}")
print(f"real: {labels[real_answer[i]]}")
plt.imshow(img_restore(misclass_imgs[i]))

### CNN with attention

#### define blocks, model

In [ ]:
class NormedConvSA_Block(eqx.Module):
    conv: eqx.nn.Conv2d
    pool: eqx.nn.MaxPool2d
    drop: eqx.nn.Dropout
    batch_norm1: eqx.nn.BatchNorm
    batch_norm2: eqx.nn.BatchNorm
    attention: eqx.nn.MultiheadAttention
    input_shape: tuple

    def __init__(
        self,
        input_shape: Sequence[int] = (3, 32, 32),
        out_channels: int = 16,
        conv_window: int = 3,
        pooling_kernel: int = 2,
        pooling_stride: int = 1,
        drop_rate: int = 0.0,
        key: jax.Array = random.key(42),
    ):
        """Block with the following layers:
        2D convolution -> 2D pooling -> RELU activation -> batch norm -> self-attention -> batch norm

        Args:
            input_shape (Sequence[int], optional): Shape of (unbatched) input. Order is Channel, Height, Width. Defaults to (3,32,32).
            out_channels (int, optional): Number of output channels. Defaults to 16.
            conv_window (int, optional): Size of 2d convolutional window. Defaults to 3.
            pooling_kernel (int, optional): Kernel for 2d max pooling. Defaults to 2.
            pooling_stride (int, optional): Stride length for pooling. Defaults to 1.
            drop_rate (int, optional): Dropout rate. Defaults to 0.0.
            key (jax.Array, optional): PRNGKey. Defaults to random.key(42).
        """          
        key1, key2 = random.split(key)
        self.conv = eqx.nn.Conv2d(
            input_shape[0], out_channels, kernel_size=conv_window, key=key1
        )
        if pooling_kernel == pooling_stride == 1:
            self.pool = eqx.nn.Identity()
        else:
            self.pool = eqx.nn.MaxPool2d(pooling_kernel, stride=pooling_stride)
        self.drop = eqx.nn.Dropout(drop_rate)

        new_H = (
            int(((input_shape[1] - conv_window) + 1 - pooling_kernel) / pooling_stride)
            + 1
        )
        self.input_shape = (out_channels, new_H, new_H)
        self.batch_norm1 = eqx.nn.BatchNorm(out_channels, "batch", momentum=0.5)
        self.batch_norm2 = eqx.nn.BatchNorm(out_channels, "batch", momentum=0.5)
        # self.batch_norm1 = eqx.nn.LayerNorm(self.input_shape[1:])
        # self.batch_norm2 = eqx.nn.LayerNorm(self.input_shape[1:])

        self.attention = eqx.nn.MultiheadAttention(
            1, new_H**2, key=key2, dropout_p=drop_rate
        )

    def __call__(self, x, key: jax.Array, state=None, train=True):
        key1, key2 = random.split(key)
        x = self.conv(x)
        x = self.pool(x)
        x = jax.nn.gelu(x)
        x = self.drop(x, key=key1, inference=not train)
        x, state = self.batch_norm1(x, state, inference=not train)
        # x = jax.vmap(self.batch_norm1)(x)
        x_q = jnp.reshape(
            x, (self.input_shape[0], self.input_shape[1] * self.input_shape[2])
        )
        attn = self.attention(x_q, x_q, x_q, key=key2, inference=not train)
        attn = jnp.reshape(
            attn, (self.input_shape[0], self.input_shape[1], self.input_shape[2])
        )
        # x = jax.vmap(self.batch_norm2)(x + attn)
        x, state = self.batch_norm2(x+attn, state, inference=not train)
        return x, state


In [ ]:
class eqx_CNN_SA_Classifier(eqx.Module):
    conv_layers: Sequence[NormedConvSA_Block]
    linear_layers: Sequence[NormedDense_Block]

    def __init__(
        self,
        input_shape: Sequence[int] = (3, 32, 32),
        num_classes: int = 10,
        conv_channels: Sequence[int] = [16],
        conv_windows: Sequence[int] = [5],
        pooling_kernel: Sequence[int] = [2],
        pooling_stride: Sequence[int] = [2],
        conv_drop: float = 0.0,
        hidden_dims: Sequence[int] = [64],
        dense_drop: float = 0.0,
        key: jax.Array = random.key(42),
    ):
        """Classifier formed by stacking convolutional+self attention and dense blocks.

    Args:
        input_shape (Sequence[int], optional): (Unbatched) shape of input. Order is Channel, Height, Width. Defaults to (3, 32, 32).
        num_classes (int, optional): Number of classes to predict. Shape of final dense output. Defaults to 10.
        conv_channels (Sequence[int], optional): Number of channels for each convolutional layer. Defaults to [16].
        conv_windows (Sequence[int], optional): Window size for each convolutional layer. Defaults to [5].
        pooling_kernel (Sequence[int], optional): Kernel size for each convolutional layer. Defaults to [2].
        pooling_stride (Sequence[int], optional): Stride length for each convolutional layer. Defaults to [2].
        conv_drop (float, optional): Dropout rate for convolutional layers. Defaults to 0.0.
        hidden_dims (Sequence[int], optional): Hidden dims for intermediate dense layers. Defaults to [64].
        dense_drop (float, optional): Dropout for dense layers. Defaults to 0.0.
        key (jax.Array, optional): PRNGKey. Defaults to random.key(42).
        """        
        def assert_equal_length(*sequences):
            # Calculate the lengths of all sequences
            lengths = [len(seq) for seq in sequences]

            # Check if all lengths are equal to the first length in the list
            assert all(
                length == lengths[0] for length in lengths
            ), f"Sequences defining ConvSA block are of different lengths: {lengths}"

        assert_equal_length(conv_channels, conv_windows, pooling_kernel, pooling_stride)

        # establish convulutional layers
        self.conv_layers = []
        key, *conv_keys = random.split(key, num=len(conv_channels) + 1)
        conv_iterator = enumerate(
            zip(conv_windows, conv_channels, pooling_kernel, pooling_stride, conv_keys)
        )
        for i, (cw, cf, pk, ps, k) in conv_iterator:
            self.conv_layers.append(
                NormedConvSA_Block(
                    input_shape=input_shape,
                    out_channels=cf,
                    conv_window=cw,
                    pooling_kernel=pk,
                    pooling_stride=ps,
                    key=k,
                    drop_rate=conv_drop,
                )
            )

            # necessary to track input shapes for each layer, follows basic convolution shape formula
            new_H = (
                int(
                    ((input_shape[1] - conv_windows[i]) + 1 - pooling_kernel[i])
                    / pooling_stride[i]
                )
                + 1
            )
            input_shape = (cf, new_H, new_H)

        # establish linear layers
        self.linear_layers = []
        key, *lin_keys = random.split(key, num=len(hidden_dims) + 1)
        input_shape = np.prod(input_shape)
        for i, (hidden_dim, k) in enumerate(zip(hidden_dims, lin_keys)):
            self.linear_layers.append(
                NormedDense_Block(
                    in_dim=input_shape, out_dim=hidden_dim, key=k, drop_rate=dense_drop
                )
            )
            input_shape = hidden_dim
        self.linear_layers.append(
            NormedDense_Block(
                in_dim=input_shape, out_dim=num_classes, key=key, drop_rate=0
            )
        )

    @eqx.filter_jit
    def __call__(self, x, key, state=None, train=True):
        subkey, *conv_keys = random.split(key, num=len(self.conv_layers) + 1)
        for layer, k in zip(self.conv_layers, conv_keys):
            x, state = layer(x, k, state=state, train=train)
        x = jnp.ravel(x)
        subkey, *lin_keys = random.split(subkey, num=len(self.linear_layers) + 1)
        for layer, k in zip(self.linear_layers, lin_keys):
            x, state = layer(x, k, state=state, train=train)
        x = jax.nn.log_softmax(x)
        return x, state


#### Set up trainer

In [ ]:
import optuna


class CNN_SAClassTrainer(EqxTrainerModule):
    def __init__(
        self,
        conv_windows: Sequence[int],
        conv_channels: Sequence[int],
        pooling_kernel: Sequence[bool],
        pooling_stride: Sequence[bool],
        hidden_dims: Sequence[int],
        input_shape: int,
        num_classes: int,
        conv_drop: float = 0.0,
        dense_drop: float = 0.0,
        trial: Any = None,
        **kwargs,
    ):
        self.trial = trial
        super().__init__(
            model_class=eqx_CNN_SA_Classifier,
            model_hparams={
                "conv_windows": conv_windows,
                "conv_channels": conv_channels,
                "pooling_kernel": pooling_kernel,
                "pooling_stride": pooling_stride,
                "input_shape": input_shape,
                "hidden_dims": hidden_dims,
                "num_classes": num_classes,
                "conv_drop": conv_drop,
                "dense_drop": dense_drop,
            },
            **kwargs,
        )

    def create_functions(self):
        def loss_function(model, x, y, key, state, train=True):
            model_out, state = jax.vmap(
                model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, key, state, train)
            # since we use log_softmax, cross entropy is just the value of our model output
            # indexed by the true value of y.
            loss = jnp.take_along_axis(model_out, jnp.expand_dims(y, 1), axis=1)
            loss = -jnp.mean(loss)

            pred_y = jnp.argmax(model_out, axis=1)
            acc = jnp.mean(y == pred_y)
            return loss, (acc, state)

        def train_step(model, opt_state, batch, key, state):
            x, y = batch
            (loss, (acc, state)), grads = eqx.filter_value_and_grad(
                loss_function, has_aux=True
            )(model, x, y, key, state)
            updates, opt_state = self.opt.update(grads, opt_state, model)
            model = eqx.apply_updates(model, updates)
            metrics = {"loss": loss, "acc": acc}
            return model, opt_state, metrics, state

        def eval_step(model, batch, key, state):
            x, y = batch
            _, (acc, _) = loss_function(model, x, y, key, state, train=False)
            return {"acc": acc}

        return train_step, eval_step

    def on_validation_epoch_end(self, epoch_idx, eval_metrics, val_loader):
        if self.trial:
            self.trial.report(eval_metrics["val/acc"], step=epoch_idx)
            if self.trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    def bind_model(self):
        """
        Returns a batched model with parameters bound to it. Enables an easier inference
        access.

        Returns:
          The model with parameters and evt. batch statistics bound to it.
        """

        def mod(x):
            model_out, state = jax.vmap(
                self.model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, self.key, self.state, False)
            return model_out

        return mod

    def vizualize_grad_norms(self, batch):
        """Plots histogram of gradient norms of model leaves"""

        def loss_function(model, x, y, key, state, train=True):
            model_out, state = jax.vmap(
                model,
                axis_name="batch",
                in_axes=(0, None, None, None),
                out_axes=(0, None),
            )(x, key, state, train)
            # since we use log_softmax, cross entropy is just the value of our model output
            # indexed by the true value of y.
            loss = jnp.take_along_axis(model_out, jnp.expand_dims(y, 1), axis=1)
            loss = -jnp.mean(loss)

            pred_y = jnp.argmax(model_out, axis=1)
            acc = jnp.mean(y == pred_y)
            return loss, (acc, state)

        def get_grad_and_path(model, batch, key, state):
            x, y = batch
            (loss, (acc, state)), grads = eqx.filter_value_and_grad(
                loss_function, has_aux=True
            )(model, x, y, key, state)
            dt = jax.tree_util.tree_leaves_with_path(grads)
            grads, names = [], []
            for (
                path,
                grad,
            ) in (
                dt
            ):  # filter basic info we don't care about (bias terms, normalization)
                if (
                    (len(grad.shape) > 1)
                    and (path[-1].name != "bias")
                    and (not path[-2].name.endswith("norm"))
                ):
                    grads.append(grad.reshape(-1))
                    names.append(path[-2].name + "_" + path[-1].name)
            return grads, names

        def viz_grads(grads, names, color="C0"):
            columns = len(grads)
            fig, ax = plt.subplots(1, columns, figsize=(columns * 3.5, 2.5))
            for g_idx, g in enumerate(grads):
                key = f"{names[g_idx]}"
                key_ax = ax[g_idx % columns]
                sns.histplot(data=g, bins=30, ax=key_ax, color=color, kde=True)
                key_ax.set_title(str(key))
                key_ax.set_xlabel("Grad magnitude")
            fig.suptitle("Gradient magnitude distributions", fontsize=14, y=1.05)
            fig.subplots_adjust(wspace=0.45)
            plt.show()
            plt.close()

        grads, names = get_grad_and_path(self.model, batch, self.key, self.state)
        viz_grads(grads, names)


#### train model

In [ ]:
trainer2 = CNN_SAClassTrainer(
    conv_windows=[5],
    conv_channels=[16],
    pooling_kernel=[2],
    pooling_stride=[2],
    hidden_dims=[64],
    input_shape=(3, 32, 32),
    num_classes=10,
    conv_drop=0.0,
    dense_drop=0.25,
    optimizer_hparams={"optimizer": "nadamw", "lr": 5e-3},
    logger_params={
        "base_log_dir": CHECKPOINT_PATH,
    },
    check_val_every_n_epoch=5,
    seed=42,
    debug=False,
)


In [ ]:
metrics2 = trainer2.train_model(train_loader, val_loader, test_loader, 5)

#### model evaluation

In [ ]:
print(f'Validation accuracy: {metrics2["val/acc"]:4.2%}')
print(f'Test accuracy: {metrics2["test/acc"]:4.2%}')

In [ ]:
batch = next(iter(train_loader))
trainer2.vizualize_grad_norms(batch)


### optuna hyperparam val example

In [ ]:
def objective(trial):
    my_train_loader, my_val_loader = create_data_loaders(
        train_set, val_set, train=[True, False], batch_size=200
    )
    trainer = CNNClassTrainer(
        conv_windows=[3, 5],
        conv_channels=[16, 32],
        pooling_kernel=[2, 2],
        pooling_stride=[2, 2],
        hidden_dims=[64],
        input_shape=(3, 32, 32),
        num_classes=10,
        conv_drop=0.0,
        dense_drop=trial.suggest_float("drop", 0.1, 0.6, log=True),
        optimizer_hparams={
            "optimizer": "nadamw",
            "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True),
            "lr": trial.suggest_float("lr", 1e-5, 5e-2, log=True),
        },
        logger_params={"base_log_dir": CHECKPOINT_PATH},
        check_val_every_n_epoch=5,
        trial=trial,
    )
    metrics = trainer.train_model(my_train_loader, my_val_loader, num_epochs=25)
    del trainer
    del my_train_loader, my_val_loader
    return metrics["val/acc"]

In [ ]:
study = optuna.create_study(
    study_name="mlp_cifar10",
    storage=f'sqlite:///{os.path.join(CHECKPOINT_PATH, trainer.config["model_class"] ,"optuna_hparam_search_eqn.db")}',
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    load_if_exists=True,
)
study.optimize(objective, n_trials=25 - len(study.trials), n_jobs=1)

In [ ]:
trial = study.best_trial
print(f"Best Validation Accuracy: {trial.value:4.2%}")
print(f"Best Params:")
for key, value in trial.params.items():
    print(f"-> {key}: {value}")

fig = optuna.visualization.plot_intermediate_values(study)
fig.show()
fig = optuna.visualization.plot_param_importances(study)
fig.show()
fig = optuna.visualization.plot_contour(study, params=["lr", "weight_decay"])
fig.show()